In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import requests

import time
from datetime import datetime

from citipy import citipy

# Import the API key.
from config import weather_api_key

In [2]:
# Weather Map API call 

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
#lat_lngs

In [4]:
coordinates = list(lat_lngs)

In [13]:
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        

In [16]:
len(cities)

740

In [17]:
# Create an endpoint URL for a city.
#city_url = url + "&q=" + "Boston"
#print(city_url)

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]
    
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Descreption": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

In [19]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descreption
0,Mataura,NZ,-46.1927,168.8643,52.48,99,100,2.84,light rain
1,Arraial Do Cabo,BR,-22.9661,-42.0278,75.38,91,100,9.24,overcast clouds
2,Busselton,AU,-33.6500,115.3333,98.04,33,57,15.95,broken clouds
3,Ushuaia,AR,-54.8000,-68.3000,42.46,70,75,23.02,broken clouds
4,Hithadhoo,MV,-0.6000,73.0833,81.64,74,100,10.09,overcast clouds
5,Georgetown,MY,5.4112,100.3354,87.73,79,20,3.00,few clouds
6,Pevek,RU,69.7008,170.3133,-19.21,100,25,3.58,scattered clouds
7,Qaanaaq,GL,77.4840,-69.3632,-13.45,56,99,8.46,overcast clouds
8,Kavieng,PG,-2.5744,150.7967,83.52,71,82,5.46,broken clouds
9,Nemuro,JP,43.3236,145.5750,25.23,60,23,14.58,few clouds


In [21]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

In [22]:
city_data_df.to_csv(output_data_file)